In [1]:
import os.path
has_train_data = os.path.isfile('train.p') 
if not has_train_data:
    from zipfile import ZipFile
    import urllib.request
    from tempfile import mktemp

    zipurl = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
    from urllib.request import urlopen
    from zipfile import ZipFile
    zipresp = urlopen(zipurl)
    tempzip = open("/tmp/tempfile.zip", "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = ZipFile("/tmp/tempfile.zip")
    zf.extractall()
    zf.close()

In [2]:
import pickle
train_data = 'train.p'
test_data = 'test.p'
with open(train_data, 'rb') as f:
    pickle_data = pickle.load(f)
    train_features = pickle_data['features']
    train_labels = pickle_data['labels']
    del pickle_data  # Free up memory
with open(test_data, 'rb') as f:
    pickle_data = pickle.load(f)
    test_features = pickle_data['features']
    test_labels = pickle_data['labels']
    del pickle_data  # Free up memory

In [3]:
def normalize_greyscale(image, a=0.1, b=0.9,grey_min=0,grey_max=255):
    return a + ( ( (image - grey_min)*(b - a) )/( grey_max - grey_min ) )
train_features = [normalize_greyscale(image) for image in train_features]
test_features = [normalize_greyscale(image) for image in test_features]

In [4]:
from sklearn.utils import shuffle
train_features, train_labels = shuffle(train_features,train_labels, random_state=123)

In [5]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)
test_labels = encoder.transform(test_labels)
# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [10]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 500
display_step = 1

# Output depth
k_output = 64

# Image Properties
image_width = 32
image_height = 32
color_channels = 3

# Convolution filter
filter_size_width = 3
filter_size_height = 3

n_classes = 43
n_hidden_layer = 2048

weights = {
    'cnn_in': tf.Variable(tf.truncated_normal([filter_size_width, filter_size_height, color_channels, k_output])),
    'cnn_out': tf.Variable(tf.random_normal([8*8*64, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}

biases = {
    'cnn_in': tf.Variable(tf.zeros(k_output)),
    'cnn_out': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
# Dropout
keep_prob = tf.placeholder(tf.float32)

# Image input
x = tf.placeholder(tf.float32,shape=[None, image_width, image_height, color_channels])
y = tf.placeholder(tf.float32, [None, 43]) # 43 classes

# Apply Convolution
conv_layer = tf.nn.conv2d(x, weights['cnn_in'], strides=[1, 2, 2, 1], padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, biases['cnn_in'])

# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

# Apply Max Pooling
conv_layer = tf.nn.max_pool(conv_layer, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')

##start neural network using the convnet output
conv_layer = tf.reshape(conv_layer, [-1, weights['cnn_out'].get_shape().as_list()[0]])
conv_layer = tf.add(tf.matmul(conv_layer, weights['cnn_out']), biases['cnn_out'])
conv_layer = tf.nn.relu(conv_layer)

# Apply Dropout
conv_layer = tf.nn.dropout(conv_layer, keep_prob)

# Output, class prediction
logits = tf.add(tf.matmul(conv_layer, weights['out']), biases['out'])


# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()


In [11]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_features)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_start = i*batch_size
            batch_features = train_features[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_features, y: batch_labels,keep_prob:0.75})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    print("Accuracy on testing data:", accuracy.eval({x: test_features, y: test_labels,keep_prob:1}))
    #print("Accuracy on training data:", accuracy.eval({x: train_features, y: train_labels,keep_prob:1}))

Epoch: 0001 cost= 1410.625538655
Epoch: 0002 cost= 29.608418868
Epoch: 0003 cost= 7.733076817
Epoch: 0004 cost= 3.733078532
Epoch: 0005 cost= 3.119309820
Epoch: 0006 cost= 2.824256225
Epoch: 0007 cost= 2.625070144
Epoch: 0008 cost= 2.424655621
Epoch: 0009 cost= 2.217741783
Epoch: 0010 cost= 2.058338341
Epoch: 0011 cost= 1.961409971
Epoch: 0012 cost= 1.842971754
Epoch: 0013 cost= 1.751816050
Epoch: 0014 cost= 1.671833054
Epoch: 0015 cost= 1.588987542
Epoch: 0016 cost= 1.532654154
Epoch: 0017 cost= 1.481198630
Epoch: 0018 cost= 1.410461531
Epoch: 0019 cost= 1.354083613
Epoch: 0020 cost= 1.322901697
Epoch: 0021 cost= 1.265570437
Epoch: 0022 cost= 1.188700920
Epoch: 0023 cost= 1.143654736
Epoch: 0024 cost= 1.121719535
Epoch: 0025 cost= 1.073348103
Optimization Finished!
Accuracy on testing data: 0.634046
